In [72]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Embedding, Dropout, Input, Concatenate, Reshape
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from google.colab import drive
drive.mount('/content/drive')
from keras.models import load_model

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
second_model = load_model('/content/drive/MyDrive/Final Election Project/second_model_2.h5')
df_2013 = pd.read_csv('/content/drive/MyDrive/Final Election Project/Complete 2013.csv')

In [74]:
def process_csv(file_path):
    # Read the CSV file

    # Extract the 'contituency' column
    constituency_column = df_2013['constituency_number']

    # Convert constituency numbers to a set to get unique values
    unique_constituencies = set(constituency_column)

    # Sort the unique constituency numbers in ascending order
    sorted_constituencies = sorted(unique_constituencies, key=lambda x: int(x.split('-')[1]))

    return sorted_constituencies

# Replace 'your_file.csv' with the actual path to your CSV file
file_path = '/content/drive/MyDrive/Final Election Project/Complete 2013.csv'
result = process_csv(file_path)

# Complete list of NA numbers from 1 to 272
complete_na_list = ['NA-{}'.format(i) for i in range(1, 273)]

# Replace 'result' with the actual result obtained from the previous code
existing_na_list = result

# Find missing NA numbers
missing_na_list = [na for na in complete_na_list if na not in existing_na_list]

# Print the missing NA numbers
print("Missing NA numbers:", missing_na_list)


Missing NA numbers: []


In [75]:
len(result)

272

In [76]:
df_2013.isnull().sum()

election_date           0
election_type           0
contest_status          0
constituency_number     0
constituency_name       0
province                0
assembly                0
voter_reg              24
validated_votes         0
votes_disq              0
candidateID             0
candidate_name          0
candidate_party         0
candidate_votes         0
candidate_share         0
candidate_rank          0
outcome                 0
dtype: int64

In [77]:
df_2013['voter_reg'].fillna(df_2013['voter_reg'].mean(), inplace=True)

In [78]:
x=df_2013['constituency_number'].unique()
len(df_2013['constituency_number'].unique())

272

In [79]:
desire_col = ['constituency_number','province','candidate_name','candidate_party','voter_reg','outcome']
data = df_2013[desire_col]

In [80]:
max_candidate_name_length = 40
max_candidate_party_length = 40

In [81]:
def tokenize_and_pad(data, max_len):
    sequences = pad_sequences(data, maxlen=max_len, padding='post')
    return sequences.tolist()

In [82]:
data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())

# Extract unique constituencies
constituencies = data['constituency_number'].unique()

<ipython-input-82-df51f43c78f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
<ipython-input-82-df51f43c78f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())


In [83]:
all_features, all_labels = [], []

for constituency_number in constituencies:
    group = data[data['constituency_number'] == constituency_number]

    features = group[['province', 'candidate_name', 'candidate_party', 'voter_reg' ]]
    labels = group['outcome'].astype('category').cat.codes

    all_features.append(features)
    all_labels.append(labels)

processed_data = pd.concat(all_features, ignore_index=True)
processed_data['outcome'] = pd.concat(all_labels, ignore_index=True)

In [84]:
# Convert text columns to sequences
tok = Tokenizer(num_words=10000)
tok.fit_on_texts(processed_data['candidate_name'].astype(str))
processed_data['candidate_name_padded'] = tok.texts_to_sequences(processed_data['candidate_name'].astype(str))

tok.fit_on_texts(processed_data['candidate_party'].astype(str))
processed_data['candidate_party_padded'] = tok.texts_to_sequences(processed_data['candidate_party'].astype(str))

In [85]:
X_train, X_test, y_train, y_test = train_test_split(processed_data[['province', 'candidate_name_padded', 'candidate_party_padded','voter_reg']], processed_data['outcome'], test_size=0.2, random_state=42)

In [86]:
from keras.layers import Bidirectional, Reshape

model3 = Sequential()
model3.add(second_model)
model3.add(Reshape((-1, 1)))
model3.add(LSTM(64, return_sequences=True))
model3.add(Dropout(0.2))
model3.add(LSTM(32, return_sequences=True))
model3.add(LSTM(32))
model3.add(Dense(16, activation='relu'))
model3.add(Dense(1, activation='tanh'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [87]:
X_train_padded = pad_sequences(X_train['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')

# Assuming max_candidate_name_length is the length of your padded sequences
timesteps = max_candidate_name_length
features = 1  # Assuming each element in the sequence is a single feature

X_train_padded = X_train_padded.reshape((X_train_padded.shape[0], timesteps, features))

In [88]:
model3.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
95/95 [==============================] - 36s 152ms/step - loss: 0.2659 - accuracy: 0.9395 - val_loss: 0.2547 - val_accuracy: 0.9326
Epoch 2/10
95/95 [==============================] - 10s 107ms/step - loss: 0.2299 - accuracy: 0.9395 - val_loss: 0.2475 - val_accuracy: 0.9326
Epoch 3/10
95/95 [==============================] - 10s 107ms/step - loss: 0.2282 - accuracy: 0.9395 - val_loss: 0.2472 - val_accuracy: 0.9326
Epoch 4/10
95/95 [==============================] - 9s 91ms/step - loss: 0.2284 - accuracy: 0.9395 - val_loss: 0.2468 - val_accuracy: 0.9326
Epoch 5/10
95/95 [==============================] - 10s 106ms/step - loss: 0.2286 - accuracy: 0.9395 - val_loss: 0.2482 - val_accuracy: 0.9326
Epoch 6/10
95/95 [==============================] - 10s 107ms/step - loss: 0.2289 - accuracy: 0.9395 - val_loss: 0.2479 - val_accuracy: 0.9326
Epoch 7/10
95/95 [==============================] - 9s 92ms/step - loss: 0.2286 - accuracy: 0.9395 - val_loss: 0.2473 - val_accuracy: 0.9326
Epo

In [89]:
#model3.save('/content/drive/MyDrive/Final Election Project/model_2013.h5')
model3.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 1)                 1388866   
                                                                 
 reshape_5 (Reshape)         (None, 1, 1)              0         
                                                                 
 lstm_15 (LSTM)              (None, 1, 64)             16896     
                                                                 
 dropout_5 (Dropout)         (None, 1, 64)             0         
                                                                 
 lstm_16 (LSTM)              (None, 1, 32)             12416     
                                                                 
 lstm_17 (LSTM)              (None, 32)                8320      
                                                                 
 dense_10 (Dense)            (None, 16)               

In [90]:
X_test_padded = pad_sequences(X_test['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')
X_test_padded = X_test_padded.reshape((X_test_padded.shape[0], timesteps, features))

# Evaluate the model on the test set
evaluation = model3.evaluate(X_test_padded, y_test)
print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

30/30 [==============================] - 1s 22ms/step - loss: 0.2132 - accuracy: 0.9450
Test Loss: 0.21322330832481384
Test Accuracy: 0.9450317025184631


In [94]:
predictions_2013 = model3.predict(X_test_padded)
predictions_2013

30/30 [==============================] - 1s 28ms/step


array([[0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103439],
       [0.06103439],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.06103441],
       [0.061